In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])
application_df.head()

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()


In [ ]:
# For each of the columsn with 10 unique values, determine the number of data points for each unique value
over_ten_uniques = application_df.drop(columns=["AFFILIATION","STATUS","USE_CASE","ORGANIZATION","INCOME_AMT","SPECIAL_CONSIDERATIONS","IS_SUCCESSFUL"])
over_ten_uniques.nunique()


In [ ]:
# For each of the columns with 10 unique values, determine the number of data points for each unique value

#number of data points for each unique value of Application Types
application_value_counts = application_df.APPLICATION_TYPE.value_counts()

#number of data points for each unique value of Classifications
application_value_counts1 = application_df.CLASSIFICATION.value_counts()

#number of data points for each unique value of Ask Amounts
application_value_counts2 = application_df.ASK_AMT.value_counts()

#printing the values for each unique value to see if the code worked
print(application_value_counts1, application_value_counts2, application_value_counts)

In [ ]:
#density chart for Classifications
application_value_counts1.plot.density()

In [ ]:
#density chart for Ask Amounts
application_value_counts2.plot.density()

In [ ]:
#density chart for Application types
application_value_counts.plot.density()

In [ ]:
#density chart for all 3 columns we included above
#density chart for Classifications
over_ten_uniques.plot.density()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_value_counts = application_df.APPLICATION_TYPE.value_counts()
application_value_counts

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
application_value_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...?
# From the chart and list of values for application types i will put anything with less than 500 count into the "Other Bin" 
replace_application = list(application_value_counts[application_value_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df.CLASSIFICATION.value_counts()
classification_value_counts

In [ ]:
#density chart for Classifications
classification_value_counts.plot.density()

In [ ]:
# Replace in dataframe
replace_classification = list(application_value_counts1[application_value_counts1 < 1000].index)

for app1 in replace_classification:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(app1,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
# Visualize the value counts of CLASSIFICATION
newclsdensity = application_df.CLASSIFICATION.value_counts()
newclsdensity.plot.density()

In [ ]:
#replacing another 
# Look at affiliation values counts for binning
Affiliation_value_counts = application_df.AFFILIATION.value_counts()
Affiliation_value_counts

In [ ]:
replace_affiliation = list(Affiliation_value_counts[Affiliation_value_counts < 100].index)

for app2 in replace_affiliation:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(app2,"Other")
    
# Check to make sure binning was successful
application_df.AFFILIATION.value_counts()

In [ ]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()


In [ ]:
# check for the number of unique values in each column
application_df[application_cat].nunique()


In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values



In [ ]:

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=70)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn_new.save("trained_applications.h5")


In [ ]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_applications.h5')

In [ ]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")